# SlideRule: Interactive Tutorial
## Combining ICESat-2 ATL06 and ATL08 with Landsat outputs

Demonstrates the use of the SlideRule science data processing service for ICESat-2 applications

### See examples, documentation and more at [slideruleearth.io](https://slideruleearth.io/)

### SlideRule APIs

- `sliderule` - the core module
- `earthdata` - functions that access CMR (NASA's Common Metadata Repository)
- `h5` - APIs for directly reading HDF5 and NetCD4 data
- `raster` - APIs for sampling supported raster datasets
- `icesat2` - APIs for processing ICESat-2 data
- `gedi` - APIs for processing GEDI data
- `io` - functions for reading and writing local files with SlideRule results
- `ipysliderule` - functions for building interactive Jupyter notebooks that interface to SlideRule

### Import and Configure SlideRule

In [ ]:
import warnings
import geopandas
import ipyleaflet
import matplotlib.pyplot as plt
from sliderule import earthdata, icesat2, ipysliderule, io, sliderule
warnings.filterwarnings('ignore') # turn off warnings for demo

# set the url for the sliderule service
icesat2.init("slideruleearth.io")

### Set options for making science data processing requests to SlideRule

SlideRule ICESat-2 APIs follow streamlined versions of the [ATL06 land ice height](https://nsidc.org/sites/default/files/documents/technical-reference/icesat2_atl03_atbd_v006.pdf) and [ATL08 land and vegetation height](https://nsidc.org/sites/default/files/documents/technical-reference/icesat2_atl08_atbd_v006_0.pdf) algorithms.

SlideRule can use different sources for photon classification before calculating the average segment height.  
This is useful for example, in cases where there may be a vegetated canopy affecting the spread of the photon returns.
- ATL03 photon confidence values, based on algorithm-specific classification types for land, ocean, sea-ice, land-ice, or inland water
- [ATL08 land and vegetation](https://nsidc.org/data/atl08) photon classification
- YAPC (Yet Another Photon Classification) photon-density-based classification

In [ ]:
# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
SRwidgets.set_atl08_defaults()
SRwidgets.VBox(SRwidgets.atl08())

### Select regions of interest for submitting to SlideRule

Create polygons or bounding boxes to set our regions of interest.  
This map is also our viewer for inspecting our SlideRule ICESat-2 data returns.

In [ ]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet('Global', center=(39, -108), zoom=11)
SRwidgets.variable.options = ['h_mean', 'h_sigma', 'h_canopy',
    'h_min_canopy', 'h_mean_canopy', 'h_max_canopy',
    'canopy_openness', 'h_te_median',
    'landcover', 'snowcover', 'solar_elevation',
    'dh_fit_dx', 'dh_fit_dy', 'rms_misfit',
    'w_surface_window_final', 'landsat.value',
    'delta_time', 'cycle', 'rgt']
plot_controls = SRwidgets.VBox([SRwidgets.variable,SRwidgets.cmap,SRwidgets.reverse])
widget_control = ipyleaflet.WidgetControl(widget=plot_controls, position='bottomright')
m.map.add(widget_control)
m.map

### Build and transmit requests to SlideRule

- SlideRule will query the [NASA Common Metadata Repository (CMR)](https://cmr.earthdata.nasa.gov/) for ATL03 data within our region of interest
- ICESat-2 ATL03 data are then accessed from the NSIDC AWS s3 bucket in `us-west-2`
- The ATL03 granules is spatially subset within SlideRule to our exact region of interest
- SlideRule then uses our specified parameters to calculate average height segments from the ATL03 data in parallel
- The completed data is streamed concurrently back and combined into a `geopandas` `GeoDataFrame` within the Python client

In [ ]:
%%time
# build sliderule parameters using latest values from widget
atl08_parms = SRwidgets.build_atl08()

# add a sampling request to access Landsat data
samples = {
    "landsat": {
        "asset": "landsat-hls",
        "closest_time": "2022-01-05T00:00:00Z", 
        "bands": ["NDVI"]
    }, 
}

# clear existing geodataframe results
elevations = [sliderule.emptyframe()]

# for each region of interest
sliderule.logger.warning('No valid regions to run') if not m.regions else None
for poly in m.regions:
    # add polygon from map to sliderule parameters
    atl08_parms["poly"] = poly
    # get the HLS catalog for the region
    catalog = earthdata.stac(short_name="HLS", polygon=poly,
        time_start="2022-01-01T00:00:00Z", time_end="2022-03-01T00:00:00Z",
        as_str=True)
    # make the request to the SlideRule (ATL08-SR and ATL06-SR) endpoint
    # and pass it the request parameters to request ATL03 Data
    # keep_id is set to True to keep track of the extent_id
    atl08 = icesat2.atl08p(atl08_parms, keep_id=True)
    # make the request to the SlideRule (ATL08-SR and ATL06-SR) endpoint
    atl06_parms = atl08_parms.copy()
    atl06_parms["samples"] = samples
    atl06 = icesat2.atl06p(atl06_parms, keep_id=True).drop(columns=['geometry'])
    # skip iteration if there is an empty dataframe
    if atl06.empty or atl08.empty:
        continue
    # merge dataframes if there are valid values
    merged = geopandas.pd.merge(atl08, atl06, on='extent_id',
        how='left', suffixes=(None,'.atl06')).set_axis(atl08.index)
    elevations.append(merged)
# concatenate the results into a single geodataframe
gdf = geopandas.pd.concat(elevations).dropna()
# clear elevations variable to save memory
elevations = None

# add the results to the map
%matplotlib inline
if gdf.empty:
    sliderule.logger.warning('No valid SlideRule returns')
else:
    # ATL06-SR and ATL08-SR fields for hover tooltip
    fields = ['h_mean', 'h_sigma', 'h_canopy',
        'h_min_canopy', 'h_mean_canopy', 'h_max_canopy',
        'landsat.value']
    gdf.leaflet.GeoData(m.map, column_name=SRwidgets.variable.value,
        cmap=SRwidgets.colormap, max_plot_points=10000,
        tooltip=True, colorbar=True, fields=fields)
    # install handlers and callbacks
    gdf.leaflet.set_observables(SRwidgets)
    gdf.leaflet.add_selected_callback(SRwidgets.atl06_click_handler)
    m.add_region_callback(gdf.leaflet.handle_region)

### Review tabular output
Can inspect the columns, number of returns and returns at the top of the `GeoDataFrame`.

See the [SlideRule ICESat-2 API documentation](https://slideruleearth.io/rtd/user_guide/ICESat-2.html#elevations) for descriptions of each column

In [ ]:
print(f'Returned {gdf.shape[0]} records')
gdf.head()

### Create plots for a single track
Select a track from the leaflet plot above by clicking on one of the plotted elevations: RGT and Cycle will automatically get populated below

In [ ]:
SRwidgets.VBox([
    SRwidgets.rgt,
    SRwidgets.ground_track,
    SRwidgets.cycle,
])

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(nrows=2, sharex=True)
# create plots
plot_kwargs = SRwidgets.plot_kwargs.copy()
plot_kwargs.pop('column_name')
gdf.icesat2.plot(ax=ax[0], kind='scatter',
    data_type='atl06', column_name='h_mean', **plot_kwargs)
gdf.icesat2.plot(ax=ax[1], kind='scatter',
    data_type='atl08', column_name='h_canopy', **plot_kwargs)

### Save data to output file

In [ ]:
display(SRwidgets.filesaver)

In [ ]:
# append sliderule api version to attributes
version = sliderule.get_version()
atl08_parms['version'] = version['icesat2']['version']
atl08_parms['commit'] = version['icesat2']['commit']
atl08_parms['lineage'] = dict(short_name='ATL03', version=earthdata.__get_version("ATL03"))
# save to file in format (HDF5 or netCDF)
io.to_file(gdf, SRwidgets.file,
    format=SRwidgets.format,
    parameters=atl08_parms,
    regions=m.regions,
    verbose=True)